### Imports

In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import manage_data

import numpy as np
import pandas as pd

import warnings
from urllib3.exceptions import InsecureRequestWarning

warnings.simplefilter("ignore", InsecureRequestWarning)

### Connecting to ArcGIS

In [2]:
# Login for the notebook running in AGOL
#gis = GIS("home")

# Login for the notebook running locally...
gis = GIS("pro")

### Variables
Currently the production layer (which is already powering the very very pretty Dashboard) is commented out and I'm grabbing a test layer instead (next block down). Will un-comment the operational layer when I'm 100% sure things aren't blowing up anymore.

In [3]:

# The Item ID of the service containing both the geometry layers and the dashboard layer
# dd is "Disaster Declarations"
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"

# Get the item at this item ID
dd_item = gis.content.get(dd_id)

# Item ID 1 is the input layer used for getting geometries
geometries_layer = dd_item.layers[1]

print(geometries_layer)

# Item ID 0 is the output layer displayed in the dashboard
# COMMENTING OUT WHILE TESTING
#dashboard_layer = dd_item.layers[0]

#print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations_Summaries_v2/FeatureServer/1">


### Get the Dashboard layer FOR TESTING ONLY

In [4]:
# ID of the item including Disaster Declarations Summaries subset FOR TESTING ONLY,
# replaces "dashboard_layer" above through duration of testing
test_id = "edb716da51bc4f7882d13d425ad08fd2"

test_item = gis.content.get(test_id)

dashboard_layer = test_item.layers[0]

print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/DisasterDeclarations_forTesting_2025only/FeatureServer/0">


### Connect to OpenFEMA API and get Disaster Declarations Summaries
* Right right right; I forgot that the API by default only returns 1000 records. I shouldn't really NEED more records than that, since the script is going to be run once per day. One thousand records should be MORE than enough. But, I now realize I need to do a little footwork to make sure I am just getting the 1000 _most recent_ records...

* Okay added a sort order to the api call to only get the most recent records by declarationDate! That should do it...

In [5]:
# Within the API URL, filter the records to return only fyDeclared to 2013 or newer.
# US Census GDBs only go back to 2013; before that it's shapefiles only
# and I refuse to touch shapefiles, at least for the scope of this project.

api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

# Plug in the URL and capture the response obj
response = requests.get(api_url)

# Convert response to JSON
data = response.json()

# Okay so after a little digging I really only need the following (leave out the metadata)
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

summaries_df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5612-CA,5612,CA,FM,2025-09-03T00:00:00.000Z,2025,Fire,2-7 FIRE,False,False,...,99009,Calaveras (County),25121,None,2025090301,9,R,2025-09-03T18:41:07.857Z,d017531813b75fc753371c26b246931d48de651e,28a1ba9f-d914-4024-9e75-4a66b5bba092
1,FM-5611-MT,5611,MT,FM,2025-08-26T00:00:00.000Z,2025,Fire,WINDY ROCK FIRE,False,False,...,99077,Powell (County),25119,None,2025082701,8,R,2025-08-28T18:01:23.160Z,29e175a73b969da6864182e703e3cb3f8d0bb32d,41329e57-2046-4196-a63d-902f3e7c923c
2,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99017,Deschutes (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,c4a190d030807595da90813aabc6ad2175917668,df7cb24f-8e5a-4c1e-923e-4c75c9ec4581
3,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99031,Jefferson (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,8b07b29243bdbba511790332bd3fa9cca0fe33fd,f0604c05-113b-449e-8e4a-f3b5076af546
4,FM-5609-HI,5609,HI,FM,2025-08-19T00:00:00.000Z,2025,Fire,KUNIA ROAD FIRE,False,False,...,99003,Honolulu (County),25114,None,2025082001,9,R,2025-08-21T18:22:16.374Z,731df26a647e5a0338177f445bab7a23b6f8d6ed,ffab7fa0-2e69-428d-b4d3-da95ca352c03


### Convert pseudo-date columns to actual date columns
I checked all the fips / code fields to ensure they're object / string type (I deleted that block while tidying up), so the offending fields remaining are the pseudo-date fields

In [6]:
date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

for dc in date_columns:
    summaries_df[dc] = pd.to_datetime(summaries_df[dc], errors="coerce")

### Add full FIPS Code (counties) and full Tribal Code columns

In [7]:
# Add to my df the fields I will need for comparison
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

print(summaries_df["fipsFullCode"].head())
print(summaries_df["fipsTribalCode"].head())

0    06009
1    30077
2    41017
3    41031
4    15003
Name: fipsFullCode, dtype: object
0    0699009
1    3099077
2    4199017
3    4199031
4    1599003
Name: fipsTribalCode, dtype: object


* Considering how I'm going to get the data from the API in shape for the comparison etc. I should just add the two additional columns I added manually for the dashboard I made first, COVID and Entity. After I add and calculate them, the comparisons will all be much easier, because I can just reference those fields for processing the data in chunks (i.e. step 1 process statewide, step 2 process counties, step 3 process tribal)

### Add & calc "COVID19" field

In [8]:
# Good lord I can't remember how to calculate any of these fields with pandas... 🤣😭
# Anyway the first one I need to calc is the COVID column, simple yes/no

summaries_df["COVID19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")


### Add & calc "Entities" field

In [9]:
# For my next trick I'll use np.select instead of np.where since to code new Entity column
# I have three possible values not just 2 / yes no / on off

entity_conditions = [
    summaries_df["designatedArea"] == "Statewide",
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] == "000"),
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] != "000")
]

entity_values = ["State or Equivalent", "Tribal Area or Equivalent", "County or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

### Groupby "femaDeclarationString", get temporal range (prelim analysis; to delete)

In [10]:
analyze = summaries_df.groupby("femaDeclarationString")["declarationDate"].agg(["min", "max"])

analyze["range"] = (analyze["max"] - analyze["min"]).dt.total_seconds() / 3600

analyze["range"].max()

0.0

### Let's just re-do the bottom four blocks with a one-liner now that ChatGPT is helping jog my memory from DSTP XD
Can you believe I forgot about isin?? And MERGE??

In [11]:
dashboard_sdf = dashboard_layer.query().sdf

In [12]:
adds_df = summaries_df[~summaries_df["femaDeclarationString"].isin(dashboard_sdf["femaDeclarationString"])]

### Build dict of primary/foreign keys
Okay now that I have a little more of the script fleshed out, tidying this up and re-doing a few comments here. Main thing I have to remember for further down the script is that THE KEYS ARE THE FIELDS IN THE GEOMETRIES.

In [13]:
# I believe I will need three different queries for my geometries layer
# I'll need to query to get state geometries, county geometries and tribal area geometries:

# Get the states on this field:
# (States we'll check first; )
state_field = "State_FIPS"

# Get the counties on this field:
county_field = "Full_FIPS"

# Crappily, in assembling the original dashboard layer I realized that
# the Declaration data may match ANY ONE of these tribal fields, so I need to check all three:
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"

# Just updated the values below to reflect the fields I've added to the Summaries df; they should be GTG now
key_fields_dict = {
    "State_FIPS": "fipsStateCode",
    "Full_FIPS": "fipsFullCode",
    "AIANNHFP1": "fipsTribalCode",
    "AIANNHFP2": "fipsTribalCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "fipsTribalCode" # What a pain in the @$$...
}

### Get geometries associated with each Dec string for all three entity types
Since this has to be done technically 5 times (there are three different keys for Tribal), a function it is
(Yeah we'll throw everything in a function later, probably, since it's just tidy, but this requires it)

In [14]:
# Create a spatial dataframe from the geometries layer
# I really only need the 
geometries_sdf = geometries_layer.query().sdf

In [15]:
state = "State or Equivalent"
county = "County or Equivalent"
tribal = "Tribal or Equivalent" 

def get_geometries(entity, field):

    # For the loop I want to make sure I'm only looking at the adds
    # that correspond to the entity level I'm interested in here
    entity_adds_df = adds_df[adds_df["Entity"] == entity]

    # NEED to specify an empty string for at least one of the suffixes here;
    # Otherwise I have no "Entity" field and my Dashboard layer and adds feature collection won't have matching fields
    merged = entity_adds_df.merge(geometries_sdf, left_on=[key_fields_dict[field]], right_on=field, suffixes=('', '_2'))

    print(merged.head())

    return merged


state_geometries = get_geometries(state, state_field)

county_geometries = get_geometries(county, county_field)

tribal1_geometries = get_geometries(tribal, tribal_field1)

tribal2_geometries = get_geometries(tribal, tribal_field2)

tribal3_geometries = get_geometries(tribal, tribal_field3)

# I only need to call for tribal three times since the foreign key is potentially in one of three fields.
# But they are all the same type of entity and after the merge the schemas should be identical, so just stack them.
tribal_geometries = pd.concat([tribal1_geometries, tribal2_geometries, tribal3_geometries])

Empty DataFrame
Columns: [femaDeclarationString, disasterNumber, state, declarationType, declarationDate, fyDeclared, incidentType, declarationTitle, ihProgramDeclared, iaProgramDeclared, paProgramDeclared, hmProgramDeclared, incidentBeginDate, incidentEndDate, disasterCloseoutDate, tribalRequest, fipsStateCode, fipsCountyCode, placeCode, designatedArea, declarationRequestNumber, lastIAFilingDate, incidentId, region, designatedIncidentTypes, lastRefresh, hash, id, fipsFullCode, fipsTribalCode, COVID19, Entity, OBJECTID, Entity_2, Full_FIPS, County_FIPS, County_Name, State_FIPS, State_Name, State_Abbreviation, Tribal_FIPS, Tribal_Basename, Tribal_Name, AIANNHFP1, AIANNHFP2, AIANNHFP3, AIANNHNS, Census_Year, Shape__Area, Shape__Length, SHAPE]
Index: []

[0 rows x 51 columns]
  femaDeclarationString  disasterNumber state declarationType  \
0            FM-5612-CA            5612    CA              FM   
1            FM-5611-MT            5611    MT              FM   
2            FM-5610-

### Good morning (whoops, afternoon) 9/7
Just inserting myself here on a grossly sunny Sunday. Need to wrap the rest of this stuff in functions and slam in ALL the entities, not just counties.

But first--can't I--shouldn't I--just smash all the tribal dfs together into one df? There's no reason not to do that right, and I wouldn't want Dec strings that applied to different tribal fields (1, 2, 3) to be added as separate rows.

In [16]:
def convert_to_feature_collection(input_df):

    if not input_df.empty:
        feature_collection = input_df.spatial.to_feature_collection()
    else: feature_collection = False

    return feature_collection

state_adds_fcol = convert_to_feature_collection(state_geometries)

county_adds_fcol = convert_to_feature_collection(county_geometries)

tribal_adds_fcol = convert_to_feature_collection(tribal_geometries)

In [17]:
remove_fields = ["OBJECTID", "Shape__Area", "Shape__Length", "SHAPE"]

dissolve_fields = [f["name"] for f in dashboard_layer.properties.fields if not f["name"] in remove_fields]

print(dissolve_fields)

['Entity', 'femaDeclarationString', 'disasterNumber', 'state', 'declarationType', 'declarationDate', 'fyDeclared', 'incidentType', 'declarationTitle', 'incidentBeginDate', 'incidentEndDate', 'disasterCloseoutDate', 'tribalRequest', 'fipsStateCode', 'declarationRequestNumber', 'lastIAFilingDate', 'incidentId', 'region', 'designatedIncidentTypes', 'COVID19']


Ah hah! Okay that's why my dissolve on all the fields is failing...in my adds, because of the merge, I have and Entity_x and Entity_y. But no Entity.

Soooo I'm pretty sure I can specify the names of those fields right? I remember doing that in DSTP...?

In [18]:
def disssolve_boundaries(feature_collection):

    if feature_collection:
        adds_dissolved = manage_data.dissolve_boundaries(feature_collection, dissolve_fields=dissolve_fields)
    else: adds_dissolved = False

    return adds_dissolved

state_adds_dissolved = disssolve_boundaries(state_adds_fcol)

county_adds_dissolved = disssolve_boundaries(county_adds_fcol)

tribal_adds_dissolved = disssolve_boundaries(tribal_adds_fcol)

{"cost": 0.004}


In [22]:
def append_features(dissolved):

    if dissolved:

        features = dissolved.query().features

        message = dashboard_layer.edit_features(features)

        return message

state_message = append_features(state_adds_dissolved)

county_message = append_features(county_adds_dissolved)

tribal_message = append_features(tribal_adds_dissolved)

print(state_message)

print(county_message)

print(tribal_message)

None
{'addResults': [{'objectId': 103, 'uniqueId': 103, 'globalId': None, 'success': True}, {'objectId': 104, 'uniqueId': 104, 'globalId': None, 'success': True}, {'objectId': 105, 'uniqueId': 105, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
None
